In [9]:
#Setting Spark with MinIO
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,FloatType
from pyspark.sql.functions import col,to_date,date_format,split,split,trim,upper,regexp_replace
import os
from pyspark.sql import SparkSession
S3_ACCESS_KEY = os.getenv("S3_ACCESS_KEY")
S3_SECRET_KEY = os.getenv("S3_SECRET_KEY")
S3_ENDPOINT = os.getenv("S3_ENDPOINT")

spark = SparkSession.builder \
    .appName("MinIOReader") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT) \
    .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .getOrCreate()

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
spark.catalog.clearCache()


In [10]:
#schema defining
Profile_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("country", StringType(), True),
    StructField("account_type", StringType(), True),
    StructField("credit_score", IntegerType(), True),
    StructField("status", StringType(), True),
    StructField("timestamp", StringType(), True)
])

merchant_schema = StructType([
    StructField("merchant_id", StringType(), True),
    StructField("merchant_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("country", StringType(), True),
    StructField("risk_level", StringType(), True),
    StructField("timestamp", StringType(), True)
])
account_schema = StructType([
    StructField("account_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("current_balance", FloatType(), True),
    StructField("currency", StringType(), True),
    StructField("last_updated", StringType(), True),
  
])



In [11]:

#reading data from the buckets
BUCKET_NAME = os.getenv("BUCKET_NAME")

profile_df=spark.read.format("json")\
    .schema(Profile_schema)\
    .load(f"s3a://{BUCKET_NAME}/bronze/profiles-folder/*.json")

merchant_df= spark.read.format("json")\
    .schema(merchant_schema)\
    .load(f"s3a://{BUCKET_NAME}/bronze/merchant-folder/*.json")

account_df= spark.read.format("json")\
    .schema(account_schema)\
    .load(f"s3a://{BUCKET_NAME}/bronze/account-folder/*.json")




In [12]:
#coalescing the dataframes to 2 partitions each
profile_df = profile_df.coalesce(2)
merchant_df = merchant_df.coalesce(2)
account_df = account_df.coalesce(2)

print("Number of partitions:", profile_df.rdd.getNumPartitions())
print("Number of partitions:", merchant_df.rdd.getNumPartitions())
print("Number of partitions:", account_df.rdd.getNumPartitions())

partition_data = profile_df.rdd.mapPartitionsWithIndex(
    lambda idx, it: [(idx, list(it))] 
).collect()

for idx, rows in partition_data:
    print(f"\n🧩 Partition {idx}: {len(rows)} sample rows")
    for r in rows:
        print(r)




Number of partitions: 2
Number of partitions: 2
Number of partitions: 2

🧩 Partition 0: 5 sample rows
Row(user_id='U0003', name='Richard Bradford', country='Sierra Leone', account_type='Savings', credit_score=738, status='active', timestamp='2025-02-26T04:28:17.385093')
Row(user_id='U0005', name='Kevin Boyle', country='American Samoa', account_type='Current', credit_score=508, status='closed', timestamp='2025-05-24T08:37:45.541514')
Row(user_id='U0006', name='Connor Bowers', country='Netherlands', account_type='Current', credit_score=365, status='active', timestamp='2025-06-22T23:39:09.027100')
Row(user_id='U0008', name='Wesley Sampson', country='Albania', account_type='Savings', credit_score=634, status='suspended', timestamp='2025-04-29T10:18:13.777981')
Row(user_id='U0001', name='Ashley Wilson PhD', country='France', account_type='Current', credit_score=622, status='closed', timestamp='2025-05-01T11:52:23.342166')

🧩 Partition 1: 5 sample rows
Row(user_id='U0002', name='Barbara Cald

In [13]:
#esnure every user_id starts with U 
profile_df = profile_df.filter(col("user_id").startswith("U"))

#transforming the timestamp column to date, month and year for profile dataframe

profile_df = profile_df.withColumn("date", to_date(col("timestamp"), "yyyy-MM-dd"))\
.withColumn("month", date_format("date", "MMMM"))\
.withColumn("year", date_format("date", "yyyy"))\
.withColumn("time",date_format(col("timestamp"), "HH:mm:ss"))\
.withColumn("country",regexp_replace(trim(col("country")), " ", ""))


profile_df = profile_df


#splitted names to first and last names
profile_df = profile_df.withColumn("first_name", split(col("name"), " ").getItem(0))
profile_df = profile_df.withColumn("last_name", split(col("name"), " ",).getItem(1))




#handling duplicates

profile_duplictes = profile_df.groupBy("user_id")\
            .count()\
            .filter("count > 1")

profile_df = profile_df.dropDuplicates(["user_id"])

profile_df = profile_df.select('user_id', 'first_name', 'last_name', 'country', 'account_type', 'credit_score', 'status', 'date', 'month', 'year','time')
profile_df.show()


+-------+----------+---------+-------------+------------+------------+---------+----------+---------+----+--------+
|user_id|first_name|last_name|      country|account_type|credit_score|   status|      date|    month|year|    time|
+-------+----------+---------+-------------+------------+------------+---------+----------+---------+----+--------+
|  U0001|    Ashley|   Wilson|       France|     Current|         622|   closed|2025-05-01|      May|2025|11:52:23|
|  U0002|   Barbara| Calderon|      Vietnam|     Current|         444|   active|2025-02-28| February|2025|14:01:00|
|  U0003|   Richard| Bradford|  SierraLeone|     Savings|         738|   active|2025-02-26| February|2025|04:28:17|
|  U0004|     Kevin|  Sanchez|   Montserrat|     Savings|         717|   active|2025-03-30|    March|2025|02:35:41|
|  U0005|     Kevin|    Boyle|AmericanSamoa|     Current|         508|   closed|2025-05-24|      May|2025|08:37:45|
|  U0006|    Connor|   Bowers|  Netherlands|     Current|         365|  

In [14]:
#ensurinq that merchant id starts with M
merchant_df = merchant_df.filter(col("merchant_id").startswith("M"))

#duplicate records 
duplicates_key_df = (
    merchant_df.groupBy("merchant_id")
    .count()
    .filter("count > 1")
    .select("merchant_id")
)

#removing the dupicates
merchant_df = merchant_df.dropDuplicates(["merchant_id"])

#trimming all string data columns

def trim_all_string_columns(merchant_df):

    string_cols = [f.name for f in merchant_df.schema.fields if f.dataType.simpleString() == "string"]
    for c in string_cols:
        merchant_df = merchant_df.withColumn(c, trim(col(c)))
    return merchant_df

merchant_df = trim_all_string_columns(merchant_df)

#tranfroming the timestamp column to date month and year for merchant data

merchant_df = merchant_df.withColumn("date", to_date(col("timestamp"), "yyyy-MM-dd"))
merchant_df = merchant_df.withColumn("month", date_format("date", "MMMM"))
merchant_df = merchant_df.withColumn("year", date_format("date", "yyyy"))
merchant_df = merchant_df.select('merchant_id', 'merchant_name', 'category', 'country', 'risk_level', 'date', 'month', 'year')


merchant_df.show()



+-----------+--------------------+----------+-------+----------+----------+---------+----+
|merchant_id|       merchant_name|  category|country|risk_level|      date|    month|year|
+-----------+--------------------+----------+-------+----------+----------+---------+----+
|      M0001|    Tucker-Gutierrez|      Fuel|    USA|      High|2025-08-06|   August|2025|
|      M0002|       Rivers-Miller|      Fuel|    USA|       Low|2025-01-16|  January|2025|
|      M0003|          White-Ruiz|Restaurant| Russia|    Medium|2025-03-14|    March|2025|
|      M0004|     Harvey and Sons|      Fuel|    USA|       Low|2025-08-06|   August|2025|
|      M0005|         White-Adams|Restaurant|  India|    Medium|2025-03-14|    March|2025|
|      M0006|     Herrera-Jenkins|Restaurant| Russia|      High|2025-06-24|     June|2025|
|      M0007|      Stout-Campbell|      Fuel| Russia|    Medium|2025-07-10|     July|2025|
|      M0008|        Wilson-Henry|Restaurant|    USA|    Medium|2025-09-09|September|2025|

In [15]:
#account id starts with A and user id starts with U
account_df = account_df.filter(col("account_id").startswith("A") & col("user_id").startswith("U")) 
                       
#extracting date month year from last_updated column
account_df = account_df.withColumn("date", to_date(col("last_updated"), "yyyy-MM-dd"))\
                    .withColumn("month", date_format("date", "MMMM")) \
                    .withColumn("year", date_format("date", "yyyy"))\
                    .withColumn("Time",date_format(col("last_updated"), "HH:mm:ss"))

# #ensuring the userid , accountid , cuurency are Uppercase

account_df = account_df.withColumn("account_id", upper(col("account_id"))) \
                       .withColumn("user_id", upper(col("user_id"))) \
                       .withColumn("currency", upper(col("currency")))
##duplicate datas
duplicate_keys = (
    account_df.groupBy("account_id")
    .count()
    .filter("count > 1")
)
account_df = account_df.dropDuplicates(["account_id"])

#filtering the account data for current balance greater than 0 and not null currency

account_df = account_df.filter((col("current_balance") > 0) & (col("currency").isNotNull()))

account_df.select('account_id', 'user_id', 'current_balance', 'currency', 'date', 'month', 'year','time','last_updated').show()

+----------+-------+---------------+--------+----------+--------+----+--------+--------------------+
|account_id|user_id|current_balance|currency|      date|   month|year|    time|        last_updated|
+----------+-------+---------------+--------+----------+--------+----+--------+--------------------+
|     A0001|  U0001|       123389.2|     INR|2025-10-08| October|2025|00:41:08|2025-10-08T00:41:...|
|     A0002|  U0002|      175134.52|     INR|2025-04-27|   April|2025|13:35:15|2025-04-27T13:35:...|
|     A0003|  U0003|       60881.74|     INR|2025-01-15| January|2025|02:53:07|2025-01-15T02:53:...|
|     A0004|  U0004|       58078.82|     INR|2025-07-27|    July|2025|06:24:20|2025-07-27T06:24:...|
|     A0005|  U0005|       15109.53|     INR|2025-02-14|February|2025|10:55:06|2025-02-14T10:55:...|
|     A0006|  U0006|      147521.55|     INR|2025-04-04|   April|2025|08:54:36|2025-04-04T08:54:...|
|     A0007|  U0007|       87322.65|     INR|2025-04-26|   April|2025|17:31:56|2025-04-26T1

In [ ]:
profile_df.write.format("parquet").mode("append").save(f"s3a://{BUCKET_NAME}/silver/processed-profiles/")
merchant_df.write.format("parquet").mode("append").save(f"s3a://{BUCKET_NAME}/silver/processed-merchants/")
account_df.write.format("parquet").mode("append").save(f"s3a://{BUCKET_NAME}/silver/processed-accounts/")
